In [1]:
%load_ext dotenv
%dotenv

In [2]:
import pandas as pd
import os
from pinecone import Pinecone,ServerlessSpec
from dotenv import load_dotenv,find_dotenv
import pinecone
from sentence_transformers import SentenceTransformer


c:\Users\omkat\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
files=pd.read_csv("course_section_descriptions.csv",encoding="ANSI")

In [4]:
files

,course_id,course_name,course_slug,course_description,course_description_short,course_technology,course_topic,course_instructor_quote,section_id,section_name,section_description
0,2,Introduction to Tableau,tableau,Tableau is now one of the most popular busines...,Teaching you how to tell compelling stories wi...,tableau,data visualization,Data scientists don’t just need to deal with d...,9,Introduction to Tableau,While Tableau is an indispensable tool in the ...
1,2,Introduction to Tableau,tableau,Tableau is now one of the most popular busines...,Teaching you how to tell compelling stories wi...,tableau,data visualization,Data scientists don’t just need to deal with d...,10,Tableau Functionalities,"In this section, you will create your first Ta..."
2,2,Introduction to Tableau,tableau,Tableau is now one of the most popular busines...,Teaching you how to tell compelling stories wi...,tableau,data visualization,Data scientists don’t just need to deal with d...,11,The Tableau Exercise,This section is a practical example that will ...
3,3,The Complete Data Visualization Course with Py...,data-visualization,The Data Visualization course is designed for ...,Teaching you how to master the art of creating...,python,data visualization,Data visualization is the face of data. Many p...,12,Introduction,"In this section, you will learn about the impo..."
4,3,The Complete Data Visualization Course with Py...,data-visualization,The Data Visualization course is designed for ...,Teaching you how to master the art of creating...,python,data visualization,Data visualization is the face of data. Many p...,13,Setting Up the Environments,"Here, we set up different environments for the..."
...,...,...,...,...,...,...,...,...,...,...,...
675,117,"Growth Analysis with SQL, Python, and Tableau",growth-analysis-with-sql-python-and-tableau,Every company aims to maximize its market pres...,Master the creation of a growth data dashboard...,tableau,data visualization,NaN,819,Case Study Overview,This part introduces the terminology and highl...
676,117,"Growth Analysis with SQL, Python, and Tableau",growth-analysis-with-sql-python-and-tableau,Every company aims to maximize its market pres...,Master the creation of a growth data dashboard...,tableau,data visualization,NaN,820,Retrieving Relevant Data from the Database,"Here, we explore the 365 database utilized in ..."
677,117,"Growth Analysis with SQL, Python, and Tableau",growth-analysis-with-sql-python-and-tableau,Every company aims to maximize its market pres...,Master the creation of a growth data dashboard...,tableau,data visualization,NaN,821,Crafting the Graphs,"Once all data sources are ready, we'll start c..."
678,117,"Growth Analysis with SQL, Python, and Tableau",growth-analysis-with-sql-python-and-tableau,Every company aims to maximize its market pres...,Master the creation of a growth data dashboard...,tableau,data visualization,NaN,822,Assembling the Dashboard,We'll organize all completed graphs across var...


In [5]:
files["unique_id"]=files["course_id"].astype(str) + '-' + files["section_id"].astype(str)

In [6]:
files["metadata"]=files.apply(lambda row:{
    "course_name":row["course_name"],
    "section_name":row["section_name"],
    "section_description":row["section_description"],
},axis=1)

In [7]:
def create_embedding(row):
    combined_text=f'''{row["course_name"]}{row["course_technology"]}
                      {row["course_description"]}{row["section_name"]}{row["section_description"]}'''
    
    return model.encode(combined_text,show_progress_bar=False)

In [8]:
model=SentenceTransformer("all-MiniLM-L6-v2")

In [9]:
files["embedding"]=files.apply(create_embedding,axis=1)

Upserting data to Pinecone

In [10]:
load_dotenv(find_dotenv(),override=True)

True

In [11]:
pc=Pinecone(api_key=os.environ.get("PINECONE_API_KEY"),environment=os.environ.get("PINECONE_ENV"))

In [12]:
index_name="my-index-3"
dimension=384
metric="cosine"

In [13]:
# if index_name in [index.name for index in pc.list_indexes()]:
#     pc.delete_index(index_name)
#     print(f"{index_name} successfully deleted.")
# else:
#     print(f"{index_name} not in index list.")

In [14]:
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric=metric,
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '467b090c1ed6b8e3416ce9c4b4e4d6dc', 'Date': 'Wed, 25 Dec 2024 12:55:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [15]:
index=pc.Index(index_name)

In [16]:
vectors_to_upsert=[(row["unique_id"],row["embedding"].tolist(),row["metadata"]) for index,row in files.iterrows()]

In [17]:
index.upsert(vectors=vectors_to_upsert)
print("Data successfully upserted to Pinecone index")

Data successfully upserted to Pinecone index


In [18]:
query="regression"
query_embedding=model.encode(query,show_progress_bar=False).tolist()

In [19]:
query_results=index.query(
    vector=[query_embedding],
    top_k=12,
    include_metadata=True
)



# index.query:
# This method is called on an object named index, which represents the vector database you are querying. It is designed to retrieve data based on vector similarity.
# vector=[query_embedding]:
# This parameter specifies the input vector for the query. The query_embedding is typically a high-dimensional representation (e.g., generated by an NLP model) that captures the semantic meaning of the query. The square brackets indicate that it is being passed as a list, which may be required by the method.
# top_k=12:
# This parameter limits the number of results returned by the query to 12. It indicates that you want to retrieve the top 12 vectors that are most similar to the provided query_embedding.
# include_metadata=True:
# This flag specifies whether to include additional metadata in the results. When set to True, the query will return not just the similar vectors but also associated metadata, which can be useful for understanding or processing the results further.


In [20]:
score_threshold=0.3

In [21]:
for match in query_results['matches']:
    if match['score']>=score_threshold:
        course_details=match.get('metadata',{})
        course_name=course_details.get('course_name','N/A')
        section_name=course_details.get('section_name','N/A')
        section_description=course_details.get('section_description','No description available')

        print(f"Matched item ID:{match['id']},Score:{match['score']}")
        print(f"Course:{course_name} \n Section :{section_name} \n Description:{section_description}")

Matched item ID:37-369,Score:0.503073692
Course:Machine Learning in Python 
 Section :Linear Regression with sklearn 
 Description:While there are many libraries that can compute a regression model, the most numerically stable one is sklearn. It is also the preferred choice of many machine learning professionals. In this section, we implement all we know about regressions in this amazing library.
Matched item ID:51-466,Score:0.466616154
Course:Machine Learning in Excel 
 Section :Multiple Linear Regression 
 Description:In section 3 you will discover multiple linear regression. We will expand on the simple linear regression techniques we covered in the previous section and discuss some practical considerations such as working with dummy variables and how to make predictions with more than one independent variable using Excel. 
Matched item ID:51-465,Score:0.463664711
Course:Machine Learning in Excel 
 Section :Simple Linear Regression 
 Description:Join us to create your first simple r